In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None


In [2]:
# Parameters
product = {"nb": "C:\\Users\\davig\\Documents\\code\\mini-projects\\movie-rec-system\\products\\eda-pipeline.ipynb"}


In [3]:
import duckdb
path = "./movies_data.duckdb"

# Setup
%reload_ext sql
%config SqlMagic.displaylimit = 20
conn = duckdb.connect(path, read_only=False)
%sql conn --alias duckdb

In [4]:
# Tables
%sql SHOW TABLES

Running query in 'duckdb'

name
genres
movie_genre_data
movies


In [5]:
%%sql 
-- # Get columns from movies
SELECT column_name
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = N'movies'

Running query in 'duckdb'

column_name
genre_ids
id
original_language
overview
popularity
release_date
title
vote_average
vote_count


In [6]:
# Get genres table
%sql select * from genres

Running query in 'duckdb'

id,name
28,Action
12,Adventure
16,Animation
35,Comedy
80,Crime
99,Documentary
18,Drama
10751,Family
14,Fantasy
36,History


In [7]:
%%sql
-- # Getting genre names per movie
WITH ExpandedGenres AS (
    SELECT 
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name
    FROM 
        (SELECT UNNEST(movies.genre_ids) as movie_genre_id, movies.id FROM movies) AS mg
    JOIN 
        movies m ON mg.id = m.id
    JOIN 
        genres g ON mg.movie_genre_id = g.id
)

SELECT
    movie_id,
    STRING_AGG(genre_name, ', ') AS genre_names
FROM 
    ExpandedGenres
GROUP BY 
    movie_id;

Running query in 'duckdb'

movie_id,genre_names
575264,"Action, Thriller, Action, Thriller, Action, Thriller, Action, Thriller, Action, Thriller, Action, Thriller, Action, Thriller, Action, Thriller"
968051,"Horror, Mystery, Thriller, Horror, Mystery, Thriller, Horror, Mystery, Thriller, Horror, Mystery, Thriller, Horror, Mystery, Thriller, Horror, Mystery, Thriller, Horror, Mystery, Thriller, Horror, Mystery, Thriller"
1034062,"Animation, Action, Fantasy, Animation, Action, Fantasy"
926393,"Action, Thriller, Crime, Action, Thriller, Crime, Action, Thriller, Crime, Action, Thriller, Crime, Action, Thriller, Crime, Action, Thriller, Crime, Action, Thriller, Crime, Action, Thriller, Crime"
554600,"Action, Drama, War, Action, Drama, War, Action, Drama, War, Action, Drama, War, Action, Drama, War, Action, Drama, War, Action, Drama, War, Action, Drama, War"
354912,"Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure, Family, Animation, Fantasy, Music, Comedy, Adventure"
1151534,"Thriller, Drama, Thriller, Drama"
507089,"Horror, Mystery, Horror, Mystery, Horror, Mystery, Horror, Mystery, Horror, Mystery, Horror, Mystery, Horror, Mystery, Horror, Mystery"
807172,"Horror, Horror, Horror, Horror, Horror, Horror, Horror, Horror"
980489,"Adventure, Action, Drama, Adventure, Action, Drama, Adventure, Action, Drama, Adventure, Action, Drama, Adventure, Action, Drama, Adventure, Action, Drama, Adventure, Action, Drama, Adventure, Action, Drama"


In [8]:
%%sql
-- # Creating table with movie and genre information
CREATE TABLE IF NOT EXISTS movie_genre_data AS
WITH ExpandedGenres AS (
    SELECT 
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name
    FROM 
        (SELECT UNNEST(movies.genre_ids) as movie_genre_id, movies.id FROM movies) AS mg
    JOIN 
        movies m ON mg.id = m.id
    JOIN 
        genres g ON mg.movie_genre_id = g.id
),
genre_names AS (
    SELECT
    movie_id,
    STRING_AGG(genre_name, ', ') AS genre_names
FROM 
    ExpandedGenres
GROUP BY 
    movie_id
)
SELECT gn.genre_names, m.id, m.original_language,
       m.overview, m.popularity, m.release_date,
       m.title, m.vote_average, m.vote_count
FROM genre_names gn
JOIN movies m
ON gn.movie_id = m.id
WHERE m.vote_count != 0

Running query in 'duckdb'

Count


In [9]:
%%sql
SELECT *
FROM movie_genre_data
LIMIT 2;

Running query in 'duckdb'

genre_names,id,original_language,overview,popularity,release_date,title,vote_average,vote_count
"Horror, Mystery",507089,en,"Recently fired and desperate for work, a troubled young man named Mike agrees to take a position as a night security guard at an abandoned theme restaurant: Freddy Fazbear's Pizzeria. But he soon discovers that nothing at Freddy's is what it seems.",7124.811,2023-10-25 00:00:00,Five Nights at Freddy's,8.5,906
Horror,807172,en,"When his daughter and her friend show signs of demonic possession, it unleashes a chain of events that forces single father, Victor Fielding, to confront the nadir of evil. Terrified and desperate, he seeks out the only person alive who's witnessed anything like it before.",2315.215,2023-10-04 00:00:00,The Exorcist: Believer,6.1,327


## Exploratory Analysis
* What are the most popular genres?
* What are the most popular movies?
* What are the most popular movies by genre?
* What are the most popular movies by year?
* What are the most popular movies by decade?
* What are the most popular movies by genre and decade?

## Most popular movies by genre
* Explode movies on genre_id
* Group by genre_id and sum popularity
* Join with genres tables to get the names
* Order genres by popularity

In [10]:
%%sql 
WITH ExpandedGenres AS (
    SELECT 
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name,
        m.popularity AS popularity,
    FROM 
        (SELECT UNNEST(movies.genre_ids) as movie_genre_id, movies.id FROM movies) AS mg
    JOIN 
        movies m ON mg.id = m.id
    JOIN 
        genres g ON mg.movie_genre_id = g.id
)

SELECT
    genre_name, ROUND(SUM(popularity)) AS TotalPopularity
FROM 
    ExpandedGenres
GROUP BY
    genre_name
ORDER BY TotalPopularity DESC
LIMIT 5;


Running query in 'duckdb'

genre_name,TotalPopularity
Thriller,131100.0
Horror,129542.0
Action,124781.0
Mystery,77466.0
Crime,55469.0


## Most popular movies

In [11]:
%%sql
SELECT title, popularity
FROM movies
ORDER BY popularity DESC
LIMIT 5;

Running query in 'duckdb'

title,popularity
Five Nights at Freddy's,7124.811
Five Nights at Freddy's,7124.811
The Nun II,2558.391
The Nun II,2558.391
Saw X,2473.173


### Do the other ones later